In [5]:
import numpy as np
#from tools.tools import *
from myastrotools.tijuca import *
import astropy.units as u
import pandas as pandas

def Run1Model(p, num_tangle = 6, num_gangle = 6):

    grid = p['name']
    ## Planet:
    planettype = p['planet_type']
    Tint = p['tint'] # Internal Temperature of your Planet in K
    Teq = ComputeTeq(p['st_teff'], p['rstar']*u.Rsun, p['au']*u.au, 
                     Ab = 0.3, fprime = 1/4) # planet equilibrium temperature 
    radius = p['pl_rad'] #Rjup
    massj = p['pl_mass']
    semi_major = p['au']
    phase = p['phase']

    ## Star:
    T_star = p['st_teff'] # K, star effective temperature
    logg = p['logg'] #logg , cgs
    metal = p['feh'] # metallicity of star
    r_star = p['rstar'] # solar radius


    ## Climate:
    nlevel = p['nlevel'] # number of plane-parallel levels in your code
    nofczns = p['nofczns'] # number of convective zones initially. Let's not play with this for now.
    nstr_upper = p['nstr_upper'] # top most level of guessed convective zone
    nstr_deep = nlevel -2 # this is always the case. Dont change this
    nstr = np.array([0,nstr_upper,nstr_deep,0,0,0]) # initial guess of convective zones
    rfacv = p['rfacv']

    ## Opacities:
    #
    planet_mh = p['mh']
    planet_mh_CtoO = p['cto']

    Teq_str = np.round(Teq, decimals=0)
    directory = f'{grid}-{planettype}-Tstar{T_star}-Rstar{r_star}-Teq{Teq_str}-sep{semi_major}-rad{radius}-mass{massj}-mh{planet_mh}-co{planet_mh_CtoO}-phase{phase}'
    #savefiledirectory = p['output_dir']+directory
    output_dir = ''
    savefiledirectory = output_dir+directory

    local_ck_path = f'/Volumes/Oy/picaso/reference/kcoeff_2020/'
    #local_ck_path = p['local_ck_path']

    planet_properties = {
        'tint':Tint, 'Teq':Teq, 'radius':radius, 'radius_unit':u.Rjup,
         'mass':massj, 'mass_unit': u.Mjup,
         'gravity': None, 'gravity_unit':None,
        'semi_major':semi_major, 'semi_major_unit': u.AU,
        'mh': planet_mh, 'CtoO':planet_mh_CtoO, 'phase':phase, 'num_tangle':num_tangle,
        'num_gangle':num_gangle, 'noTiOVO':p['noTiOVO'], 'planet_mh_str':p['mh_str'],
        'local_ck_path':local_ck_path
    }

    star_properties = {
        'Teff':T_star, 'logg':logg, 'mh':metal, 'radius':r_star
    }

    climate_run_setup = {'climate_pbottom':p['p_bottom'],
            'climate_ptop':p['p_top'],
            'nlevel':nlevel, 'nofczns':nofczns, 'nstr_upper':nstr_upper,
            'nstr_deep':nstr_deep, 'rfacv':rfacv
    }
    #opa_file = p['opa_file']
    opa_file = None
    wave_range = [float(p['wave_range'].split(',')[0].replace('[','')),
              float(p['wave_range'].split(',')[1].replace(' ','').replace(']',''))]
    spectrum_setup = {'opacity_db':opa_file,
                      'wave_range':wave_range,
                      'calculation':'reflected', 'R':150
                     }

    if p['guess'] == 'guillot':
        use_guillotpt = True


    cj = MakeModelCloudFreePlanet(planet_properties, 
                            star_properties,
                            use_guillotpt = True,
                            cdict = climate_run_setup,
                            compute_spectrum = True,
                            specdict = spectrum_setup,
                            savefiledirectory = savefiledirectory
                 )
    import time
    with open(savefiledirectory+'/terminal_output.txt','r') as f:
        z = f.read()
        k = open('ReflectXGasGiantRunReport.txt','a')
        t = time.localtime()
        outtime = str(t.tm_year)+'-'+str(t.tm_mon)+'-'+str(t.tm_mday)+'-'+str(t.tm_hour)+':'+str(t.tm_min)+':'+str(t.tm_sec)
        if 'YAY ! ENDING WITH CONVERGENCE' in z:
            k.write(savefiledirectory + ' ' +outtime + '  converged \n')
        else:
            k.write(savefiledirectory + ' ' +outtime + '  FAILED \n')
        k.close()
        
    return savefiledirectory, cj

def RunGrid(sheet_id='11u2eirdZcWZdjnKFn3vzqbCtKCodstP-KnoGXC5FdR8', 
             sheet_name='GasGiantsBaseModels', n_jobs = 3):
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&gid={sheet_name}"
    k = open('ReflectXGasGiantRunReport.txt','w')
    k.close()
    p = pd.read_csv(url)
    p = p.dropna(axis=1, how='all')
    savefiledirectory, cj = Run1Model(p.loc[0])
    import picaso.justdoit as jdi
    jdi.Parallel(n_jobs=n_jobs)(jdi.delayed(Run1Model)(p.loc[i]) for i in range(len(p)))
        
    return p


In [9]:
def GetP(sheet_id='11u2eirdZcWZdjnKFn3vzqbCtKCodstP-KnoGXC5FdR8', 
             sheet_name='GasGiantsBaseModels'):
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&gid={sheet_name}"
    p = pd.read_csv(url)
    p = p.dropna(axis=1, how='all')
    return p

p = GetP()
p = p.loc[:5]
p

,name,planet_type,output_dir,st_teff,rstar,feh,logg,pl_mass,pl_rad,mh,...,nstr_deep,nofczns,p_bottom,p_top,guess,local_ck_path,opa_file,wave_range,calculation,R
0,ReflectXGasGiantModelGrid,Neptune,/data2/lpearce/ReflectXGasGiantModelGrid/,3000.0,0.2,1.0,5.0,0.05,0.35,100.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0
1,ReflectXGasGiantModelGrid,Neptune,/data2/lpearce/ReflectXGasGiantModelGrid/,3000.0,0.2,1.0,5.0,0.05,0.35,100.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0
2,ReflectXGasGiantModelGrid,Neptune,/data2/lpearce/ReflectXGasGiantModelGrid/,3000.0,0.2,1.0,5.0,0.05,0.35,100.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0
3,ReflectXGasGiantModelGrid,Neptune,/data2/lpearce/ReflectXGasGiantModelGrid/,3000.0,0.2,1.0,5.0,0.05,0.35,100.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0
4,ReflectXGasGiantModelGrid,Neptune,/data2/lpearce/ReflectXGasGiantModelGrid/,3000.0,0.2,1.0,5.0,0.05,0.35,100.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0
5,ReflectXGasGiantModelGrid,Neptune,/data2/lpearce/ReflectXGasGiantModelGrid/,3000.0,0.2,1.0,5.0,0.05,0.35,100.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0


In [10]:
p['local_ck_path'][0]

'/data/kcoeff_2020_v3/'

In [7]:
itervals = {}
for icol in p.columns:
    vals = p.loc[:,icol].dropna().values
    if len(vals) > 1:
        itervals[icol] = vals


In [8]:
import picaso.justdoit as jdi
jdi.Parallel(n_jobs=3)(jdi.delayed(Run1Model)(p.loc[i]) for i in range(len(p)))

mkdir: ReflectXGasGiantModelGrid-Neptune-Tstar3000-Rstar0.2-Teq187.0-sep0.1-rad0.35-mass0.05-mh100-co1.5-phase0: File exists
mkdir: mkdir: ReflectXGasGiantModelGrid-Neptune-Tstar3000-Rstar0.2-Teq187.0-sep0.1-rad0.35-mass0.05-mh100-co0.5-phase45ReflectXGasGiantModelGrid-Neptune-Tstar3000-Rstar0.2-Teq187.0-sep0.1-rad0.35-mass0.05-mh100-co0.5-phase0: : File exists
File exists
mkdir: ReflectXGasGiantModelGrid-Neptune-Tstar3000-Rstar0.2-Teq187.0-sep0.1-rad0.35-mass0.05-mh100-co1.5-phase45: File exists


[('ReflectXGasGiantModelGrid-Neptune-Tstar3000-Rstar0.2-Teq187.0-sep0.1-rad0.35-mass0.05-mh100-co0.5-phase0',
  (<picaso.justdoit.inputs at 0x7f869d87a6a0>,
   {'pressure': array([1.00000000e-06, 1.22712524e-06, 1.50583635e-06, 1.84784980e-06,
           2.26754313e-06, 2.78255940e-06, 3.41454887e-06, 4.19007911e-06,
           5.14175183e-06, 6.30957344e-06, 7.74263683e-06, 9.50118507e-06,
           1.16591440e-05, 1.43072299e-05, 1.75567629e-05, 2.15443469e-05,
           2.64376119e-05, 3.24422608e-05, 3.98107171e-05, 4.88527357e-05,
           5.99484250e-05, 7.35642254e-05, 9.02725178e-05, 1.10775685e-04,
           1.35935639e-04, 1.66810054e-04, 2.04696827e-04, 2.51188643e-04,
           3.08239924e-04, 3.78248991e-04, 4.64158883e-04, 5.69581081e-04,
           6.98947321e-04, 8.57695899e-04, 1.05250029e-03, 1.29154967e-03,
           1.58489319e-03, 1.94486244e-03, 2.38658979e-03, 2.92864456e-03,
           3.59381366e-03, 4.41005945e-03, 5.41169527e-03, 6.64082785e-03,
      